In [1]:
"""import tarfile

# Ruta del archivo tar que deseas descomprimir
archivo_tar = 'datasets/yelp_dataset.tar'

# Ruta donde deseas extraer el contenido del archivo tar
ruta_destino = 'datasets/'

# Abrir el archivo tar en modo de lectura
with tarfile.open(archivo_tar, 'r') as archivo:
    archivo.extractall(path=ruta_destino)

print(f"Archivo {archivo_tar} descomprimido en {ruta_destino}")"""


Archivo datasets/yelp_dataset.tar descomprimido en datasets/


In [1]:
import pandas as pd

# Función para leer y procesar business.json
def process_business(file_name):
    chunks = pd.read_json(file_name, lines=True, chunksize=10000) # Puedes ajustar el chunksize según tus necesidades
    df_list = []
    for chunk in chunks:
        df_list.append(chunk)
    return pd.concat(df_list, ignore_index=True)

business_ny_df = process_business('YELP_DATASETS_ORIGINALES\yelp_academic_dataset_business.json')

business_ny_df.head(1)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None


In [2]:
unique_states = business_ny_df['state'].unique()
print(unique_states)


['CA' 'MO' 'AZ' 'PA' 'TN' 'FL' 'IN' 'LA' 'AB' 'NV' 'ID' 'DE' 'IL' 'NJ'
 'NC' 'CO' 'WA' 'HI' 'UT' 'TX' 'MT' 'MI' 'SD' 'XMS' 'MA' 'VI' 'VT']


In [3]:
# Luego de obtener los estados únicos como ya lo hiciste:
unique_states = business_ny_df['state'].unique()

# Agrupa por estado y cuenta el número de filas para cada uno
state_counts = business_ny_df.groupby('state').size().sort_values(ascending=False)

print(state_counts)


state
PA     34039
FL     26330
TN     12056
IN     11247
MO     10913
LA      9924
AZ      9912
NJ      8536
NV      7715
AB      5573
CA      5203
ID      4467
DE      2265
IL      2145
TX         4
CO         3
HI         2
MA         2
WA         2
NC         1
SD         1
MT         1
MI         1
UT         1
VI         1
VT         1
XMS        1
dtype: int64


In [4]:
import pandas as pd

# Función para leer y procesar business.json
def process_business(file_name):
    chunks = pd.read_json(file_name, lines=True, chunksize=10000) # Puedes ajustar el chunksize según tus necesidades
    df_list = []
    for chunk in chunks:
        df_list.append(chunk[chunk['state'] == 'FL'])
    return pd.concat(df_list, ignore_index=True)

business_ny_df = process_business('YELP_DATASETS_ORIGINALES\yelp_academic_dataset_business.json')

# Función para leer y procesar review.json por chunks y hacer merge
def process_review_and_merge(file_name, business_df):
    chunks = pd.read_json(file_name, lines=True, chunksize=10000) # Puedes ajustar el chunksize según tus necesidades
    merged_list = []
    for chunk in chunks:
        merged_chunk = pd.merge(business_df, chunk, on='business_id', how='left')
        merged_list.append(merged_chunk)
    return pd.concat(merged_list, ignore_index=True)

merged_df = process_review_and_merge('YELP_DATASETS_ORIGINALES\yelp_academic_dataset_review.json', business_ny_df)


In [5]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19073574 entries, 0 to 19073573
Data columns (total 22 columns):
 #   Column        Dtype         
---  ------        -----         
 0   business_id   object        
 1   name          object        
 2   address       object        
 3   city          object        
 4   state         object        
 5   postal_code   object        
 6   latitude      float64       
 7   longitude     float64       
 8   stars_x       float64       
 9   review_count  int64         
 10  is_open       int64         
 11  attributes    object        
 12  categories    object        
 13  hours         object        
 14  review_id     object        
 15  user_id       object        
 16  stars_y       float64       
 17  useful        float64       
 18  funny         float64       
 19  cool          float64       
 20  text          object        
 21  date          datetime64[ns]
dtypes: datetime64[ns](1), float64(7), int64(2), object(12)
memory us

In [6]:
merged_df.shape

(19073574, 22)

In [7]:
dict_columns = [col for col in merged_df.columns if merged_df[col].apply(type).eq(dict).any()]
print(f"Columnas con diccionarios: {dict_columns}")


Columnas con diccionarios: ['attributes', 'hours']


In [8]:
for col in dict_columns:
    merged_df[col] = merged_df[col].astype(str)


In [9]:
# Eliminar todas las filas duplicadas
merged_df.drop_duplicates(inplace=True)


In [10]:
pd.set_option('display.max_columns', None)  # Muestra todas las columnas
pd.set_option('display.max_rows', None)     # Muestra todas las filas
pd.set_option('display.max_colwidth', None)   # Muestra todo el contenido de una celda

# Encuentra el índice del primer duplicado
index_of_first_duplicate = merged_df.duplicated().idxmax()

if merged_df.duplicated().any():
    # Muestra la fila duplicada y la fila original que fue duplicada
    display_rows = merged_df[merged_df.duplicated(keep=False)].iloc[[index_of_first_duplicate - 1, index_of_first_duplicate]]
    
else:
    print("No hay duplicados en el DataFrame.")

display_rows


No hay duplicados en el DataFrame.


NameError: name 'display_rows' is not defined

In [11]:

# Obtener número total de filas duplicadas en el dataframe
total_duplicados = merged_df.duplicated().sum()

print(f"Total de filas duplicadas: {total_duplicados}\n")


Total de filas duplicadas: 0



In [12]:

for column in merged_df.columns:
    # Filtrar filas con NaN, None, cadenas vacías o NaT
    condition = merged_df[column].isin([None, "", pd.NaT]) | merged_df[column].isnull()
    nulos = condition.sum()
    print(f"Columna: {column}, Datos no válidos (NaN/NaT/None/''): {nulos}")

Columna: business_id, Datos no válidos (NaN/NaT/None/''): 0
Columna: name, Datos no válidos (NaN/NaT/None/''): 0
Columna: address, Datos no válidos (NaN/NaT/None/''): 13427
Columna: city, Datos no válidos (NaN/NaT/None/''): 0
Columna: state, Datos no válidos (NaN/NaT/None/''): 0
Columna: postal_code, Datos no válidos (NaN/NaT/None/''): 102
Columna: latitude, Datos no válidos (NaN/NaT/None/''): 0
Columna: longitude, Datos no válidos (NaN/NaT/None/''): 0
Columna: stars_x, Datos no válidos (NaN/NaT/None/''): 0
Columna: review_count, Datos no válidos (NaN/NaT/None/''): 0
Columna: is_open, Datos no válidos (NaN/NaT/None/''): 0
Columna: attributes, Datos no válidos (NaN/NaT/None/''): 0
Columna: categories, Datos no válidos (NaN/NaT/None/''): 152
Columna: hours, Datos no válidos (NaN/NaT/None/''): 0
Columna: review_id, Datos no válidos (NaN/NaT/None/''): 26330
Columna: user_id, Datos no válidos (NaN/NaT/None/''): 26330
Columna: stars_y, Datos no válidos (NaN/NaT/None/''): 26330
Columna: usefu

In [13]:
# Guardar en CSV
merged_df.to_csv('FLORIDA_YELP_CRUDO.csv', index=False)

# Guardar en Parquet (necesitas tener la biblioteca 'pyarrow' o 'fastparquet')
merged_df.to_parquet('FLORIDA_YELP_CRUDO.parquet', index=False)


In [15]:
merged_df.shape

(1187875, 22)